# Problem statement

The Union Chemical Sri Lanka operates five production facilities: Alpha, Beta, Gamma, Delta, 
and Epsilon. These facilities can produce four primary chemicals: Xylenol, Methacrylate, 
Ethanolamine, and Butanol. The company has outlined the resource allocation for each facility 
based on their production capabilities, raw material availability and energy constraints for the 
upcoming production cycle. 

| Facility Name | Production Capacity (’000 tons) | Energy Limit (’000 MWh) | Raw Material A (’000 tons) | Raw Material B (’000 tons) | Raw Material C (’000 tons) |
| ------------- | ------------------------------- | ----------------------- | -------------------------- | -------------------------- | -------------------------- |
| Alpha         | 35                              | 50                      | 20                         | 15                         | 10                         |
| Beta          | 30                              | 45                      | 18                         | 20                         | 12                         |
| Gamma         | 15                              | 20                      | 10                         | 9                          | 8                          |
| Delta         | 25                              | 30                      | 15                         | 12                         | 15                         |
| Epsilon       | 20                              | 25                      | 12                         | 14                         | 10                         |


| Chemical     | Max Sales (’000 tons) | Energy Req (MWh/ton) | A Req (tons/ton) | B Req (tons/ton) | C Req (tons/ton) | Profit (’000 Rs) |
| ------------ | --------------------- | -------------------- | ---------------- | ---------------- | ---------------- | ---------------- |
| Xylenol      | 70                    | 0.9                  | 2.0              | 1.0              | 0.5              | 6                |
| Methacrylate | 55                    | 1.3                  | 1.5              | 2.0              | 0.8              | 11               |
| Ethanolamine | 45                    | 1.6                  | 1.0              | 1.2              | 2.0              | 16               |
| Butanol      | 50                    | 1.1                  | 0.8              | 0.6              | 1.5              | 9                |


# Linear Programming Model

The linear programming model associated with the case study if:

**Total chemical production at Beta facility must not exceed twice the total chemical production at Epsilon 
facility.**

 **Decision Variables:**

Let

$$
x_{ij} = \text{Production of chemical } j \text{ (in '000 tons) at facility } i
$$

where:

* $i \in \{\text{Alpha, Beta, Gamma, Delta, Epsilon}\}$
* $j \in \{\text{Xylenol, Methacrylate, Ethanolamine, Butanol}\}$

---

**Objective Function:**

Maximize total profit:

$$
\text{Maximize } Z = \sum_{i,j} \text{Profit}_j \cdot x_{ij}
$$

| Chemical     | Profit ('000 Rs/ton) |
| ------------ | -------------------- |
| Xylenol      | 6                    |
| Methacrylate | 11                   |
| Ethanolamine | 16                   |
| Butanol      | 9                    |

---

**Subject to Constraints:**

 1. **Production Capacity per Facility:**

$$
\sum_j x_{ij} \leq \text{Capacity}_i \quad \forall i
$$

 2. **Energy Usage per Facility:**

$$
\sum_j \text{Energy}_j \cdot x_{ij} \leq \text{EnergyLimit}_i \quad \forall i
$$

 3. **Raw Material Limits per Facility:**

$$
\sum_j \text{A}_j \cdot x_{ij} \leq \text{A}_i \quad \forall i
$$

$$
\sum_j \text{B}_j \cdot x_{ij} \leq \text{B}_i \quad \forall i
$$

$$
\sum_j \text{C}_j \cdot x_{ij} \leq \text{C}_i \quad \forall i
$$

 4. **Sales Limits per Chemical:**

$$
\sum_i x_{ij} \leq \text{MaxSales}_j \quad \forall j
$$

 5. **Special Constraint:**

$$
\sum_j x_{\text{Beta},j} \leq 2 \cdot \sum_j x_{\text{Epsilon},j}
$$

 6. **Non-negativity:**

$$
x_{ij} \geq 0 \quad \forall i,j
$$

---



]

In [1]:
from scipy.optimize import linprog
import pandas as pd

# Facilities and Chemicals
facilities = ["Alpha", "Beta", "Gamma", "Delta", "Epsilon"]
chemicals = ["Xylenol", "Methacrylate", "Ethanolamine", "Butanol"]

# Profit per ton (objective)
profit_per_ton = {"Xylenol": 6, "Methacrylate": 11, "Ethanolamine": 16, "Butanol": 9}
c = [-profit_per_ton[ch] for f in facilities for ch in chemicals]  # Negative for maximization

# Resource capacities per facility
prod_cap = {"Alpha": 35, "Beta": 30, "Gamma": 15, "Delta": 25, "Epsilon": 20}
energy_cap = {"Alpha": 50, "Beta": 45, "Gamma": 20, "Delta": 30, "Epsilon": 25}
A_cap = {"Alpha": 20, "Beta": 18, "Gamma": 10, "Delta": 15, "Epsilon": 12}
B_cap = {"Alpha": 15, "Beta": 20, "Gamma": 9, "Delta": 12, "Epsilon": 14}
C_cap = {"Alpha": 10, "Beta": 12, "Gamma": 8, "Delta": 15, "Epsilon": 10}

# Requirements per ton of chemical
energy_req = {"Xylenol": 0.9, "Methacrylate": 1.3, "Ethanolamine": 1.6, "Butanol": 1.1}
A_req = {"Xylenol": 2, "Methacrylate": 1.5, "Ethanolamine": 1, "Butanol": 0.8}
B_req = {"Xylenol": 1, "Methacrylate": 2, "Ethanolamine": 1.2, "Butanol": 0.6}
C_req = {"Xylenol": 0.5, "Methacrylate": 0.8, "Ethanolamine": 2, "Butanol": 1.5}

# Max sales per chemical
max_sales = {"Xylenol": 70, "Methacrylate": 55, "Ethanolamine": 45, "Butanol": 50}

# Index map for variables
index = {(f, ch): i * len(chemicals) + j for i, f in enumerate(facilities) for j, ch in enumerate(chemicals)}

# Constraints
A_ub, b_ub = [], []

# Production, energy, raw material constraints per facility
for f in facilities:
    A_ub.append([1 if f2 == f else 0 for f2 in facilities for ch in chemicals])
    b_ub.append(prod_cap[f])
    A_ub.append([energy_req[ch] if f2 == f else 0 for f2 in facilities for ch in chemicals])
    b_ub.append(energy_cap[f])
    A_ub.append([A_req[ch] if f2 == f else 0 for f2 in facilities for ch in chemicals])
    b_ub.append(A_cap[f])
    A_ub.append([B_req[ch] if f2 == f else 0 for f2 in facilities for ch in chemicals])
    b_ub.append(B_cap[f])
    A_ub.append([C_req[ch] if f2 == f else 0 for f2 in facilities for ch in chemicals])
    b_ub.append(C_cap[f])

# Max sales constraints per chemical
for ch in chemicals:
    A_ub.append([1 if ch2 == ch else 0 for f in facilities for ch2 in chemicals])
    b_ub.append(max_sales[ch])

# Special constraint: Beta ≤ 2 × Epsilon
row = [1 if f == "Beta" else -2 if f == "Epsilon" else 0 for f in facilities for ch in chemicals]
A_ub.append(row)
b_ub.append(0)

# Bounds for all variables
bounds = [(0, None)] * len(c)

# Solve
res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Output
if res.success:
    print(f"✅ Optimal Profit: Rs. {round(-res.fun, 2):,.2f} ('000 Rs)")
    print("📦 Optimal Production Plan:")
    for (f, ch), idx in index.items():
        val = res.x[idx]
        if val > 1e-3:
            print(f"{f:8s} - {ch:14s} : {val:.2f} tons")
else:
    print("❌ Optimization failed:", res.message)


✅ Optimal Profit: Rs. 551.97 ('000 Rs)
📦 Optimal Production Plan:
Alpha    - Methacrylate   : 5.92 tons
Alpha    - Ethanolamine   : 2.63 tons
Beta     - Methacrylate   : 8.42 tons
Beta     - Ethanolamine   : 2.63 tons
Gamma    - Methacrylate   : 2.76 tons
Gamma    - Ethanolamine   : 2.89 tons
Delta    - Methacrylate   : 1.97 tons
Delta    - Ethanolamine   : 6.71 tons
Epsilon  - Methacrylate   : 5.26 tons
Epsilon  - Ethanolamine   : 2.89 tons
